### C# Interactive Notebook - Carbon Example No.1

This example notebook demonstrates the basic code required to use the Carbon cross-tabulation libraries to create a report. This notebook is a good place to start if you haven't used Carbon before. The code is separated into cells with their own explanations. In other notebooks, the repetitive *startup* code explained once in this notebook is simply placed in the first cell with the assumption that its purpose is understood.

---

Reference the public 'Tables' package which is used for Carbon cross-tabulation report generation. This reference will download the 'Tables' package and its child dependency packages and put them in a cache if they are not already there.

The using statements below are the ones required for typical processing of Carbon reports.

In [2]:
#r "nuget: RCS.Carbon.Tables"
using RCS.Carbon.Tables;
using RCS.Carbon.Variables;
using RCS.Carbon.Shared;
using RCS.Carbon.Licensing.Shared;

Installed Packages RCS.Carbon.Tables, 9.1.10

Create an instance of a `CrossTabEngine` class which publishes most of the public Carbon API related to processing jobs, variables and reports. An application will normally keep one instance of the engine alive for the application lifetime.

In [3]:
var engine = new CrossTabEngine();

A licence must be registered in the engine before it can perform any useful work. The `GetFreeLicence` method retrieves the special *free* licence that can be used for evaluation. An instance of a `LicenceInfo` class is returned which contains account information which may be of interest in more advanced scenarios. Some of the licence properties are displayed.

Note that **await** is used on the `GetFreeLicence` call because it makes an asynchronous call to a web service.

In [4]:
LicenceInfo licinfo = await engine.GetFreeLicence();
Console.WriteLine($"Account Id ........... {licinfo.Id}");
Console.WriteLine($"Account Name ......... {licinfo.Name}");
Console.WriteLine($"Account Email ........ {licinfo.Email}");
string rolejoin = string.Join(',', licinfo.Roles);
Console.WriteLine($"Account Roles ........ {rolejoin}");
string vtjoin = string.Join(',', licinfo.VartreeNames);
Console.WriteLine($"Account Vartrees ..... {vtjoin}");
string custjoin = string.Join(',', licinfo.Customers.Select(c => c.Name));
Console.WriteLine($"Account Vartrees ..... {custjoin}");

Account Id ........... 10000010
Account Name ......... free
Account Email ........ GitHub Example App
Account Roles ........ cache(7),free
Account Vartrees ..... 
Account Vartrees ..... rcsruby,rcspublic


The Carbon cross-tabulation engine can open *jobs* in the local filesystem or in cloud storage. A *job* is a collection of related reports and data. Just like files can be organised into folders, Carbon reports can be organised and managed in *jobs*. For this demonstration, a public cloud job is opened. This is one of many public demonstration cloud jobs published by [Red Centre Software](https://www.redcentresoftware.com/).

The Carbon engine normally has one job open and active at any time. `OpenJob` can be called as needed to close the previous job and open a new one.

In [5]:
engine.OpenJob("rcsruby", "demo");

A cross-tabulation report can now be generated for the open job.

Create an instance of an `XSpecProperties` class which can provide advanced control over how a report is generated. A default class instance using all default values is satisfactory.

Create an instance of an `xDisplayProperties` class which can provide detailed control over report appearance. A single property is set to specify that the report should be genrated in CSV (comma separated values) format. Other default values are satisfactory.

Call `GenTab` with a specified title, top variable "Age" and side variable "Region". The returned string is a set of text lines joined with a newline (\n) character.

In [6]:
var sprops = new XSpecProperties();
var dprops = new XDisplayProperties();
dprops.Output.Format = XOutputFormat.CSV;
string report = engine.GenTab("Demo of Age × Region", "Age", "Region", null, null, sprops, dprops);
Console.WriteLine(report);

Name: Demo of Age × Region
Top: Age
Side: Region
Filter: 
Weight: 
Status: 
,15-25,26-35,36-50,51-65,65+,Under 35,Over 35,Under 50,Over 50
NE,41,53,44,45,55,94.00,144.00,138.00,100.00
SE,43,48,49,52,46,91.00,147.00,140.00,98.00
SW,48,38,42,67,52,86.00,161.00,128.00,119.00
NW,40,58,68,58,53,98.00,179.00,166.00,111.00
East,84,101,93,97,101,185.00,291.00,278.00,198.00
West,88,96,110,125,105,184.00,340.00,294.00,230.00
North,81,111,112,103,108,192.00,323.00,304.00,211.00
South,91,86,91,119,98,177.00,308.00,268.00,217.00



There are many text output formats available such CSV, TSV, XML and HTML. Special output formats include XLSX (Excel workbook), JSON in the exact format required for [pandas](https://pypi.org/project/pandas/) data processing, and [DataFrames](https://learn.microsoft.com/en-us/dotnet/api/microsoft.data.analysis.dataframe?view=ml-dotnet-preview) usable by Microsoft data analysis libraries.

When all processing is complete, the `CloseJob` method should be called to release resouces held by the job.

In [7]:
bool closed = engine.CloseJob();
Console.WriteLine($"Job closed -> {closed}");

Job closed -> True
